# RMJ Parsing | Level of difficulty : 1 out of 10

### Import modules

In [ ]:
from bs4 import BeautifulSoup
from random import choice
from fake_useragent import UserAgent

from selenium import webdriver
import csv
import time
import os
import wget
import requests

print("Modules loaded successfully")

### Variables

In [ ]:
login_url    = "https://www.rmj.ru/personal/"
archive_url  = "https://www.rmj.ru/archive/?PAGEN_1={}" # in curly brackets page number
login        = "almasgaara@mail.ru"
password     = "X2n5mP4Ngq"
link_array   = []
start_page   = 1
end_page     = 17
titles       = []
categories   = []
pdfs         = []
result       = {}

print("Variables loaded successfully")

### Functions

In [ ]:
def get_data(url):
    driver.get(url)
    title    = get_element('/html/body/div[1]/div[1]/div/div[4]/div[2]/div[3]/div[1]/div/div[2]/div[1]', 0) 
    category = get_element('/html/body/div[1]/div[1]/div/div[4]/div[2]/h1', 0)
    pdf      = get_element('/html/body/div[1]/div[1]/div/div[4]/div[2]/div[3]/div[1]/div/div[2]/div[3]/a', 1)
    return {
        "title"    : title,
        "category" : category,
        "pdf"      : pdf
    }

def get_element(xpath, mode):
    element = None
    try: 
        if(mode == 0):
            element = driver.find_element_by_xpath(xpath).text
        elif(mode == 1):
            element = driver.find_element_by_xpath(xpath).get_attribute("href")
    except : pass
    return element

def get_proxy():
    proxies = []
    proxyFile = open("proxies.content.txt", "r")
    soup = BeautifulSoup(proxyFile.read(), 'html.parser')
    tr = soup.findAll('td', attrs={'class': 'tdl'})
    for td in tr:
        proxies.append(td.text)
        proxyFile.close()
    return {'http//': choice(proxies)}

def get_user_agent():
    return { 'User-Agent': UserAgent().chrome }

def get_links(html):
    links = []
    try:
        blocks = html.findAll('div', attrs={'class': 'block'})
        for block in blocks:
            images = block.findAll('div', attrs={'class': 'image'})
            for img in images:
                links.append("https://www.rmj.ru" + img.find('a')["href"])
    except:
        print("Error")
    return links

### Get all links from archive

In [ ]:
with open("links.txt", "w") as file:
    for page in range(start_page, end_page+1):
        url = archive_url.format(page)
        print(f"parsing...{url} :")
        response = requests.get(url=url,
            headers=get_user_agent(), proxies=get_proxy())
        html = response.content
        soup = BeautifulSoup(html, 'html.parser')
        for link in get_links(soup):
            file.write(link + "\n")
            print("Success!")
            file.close()

### Bypass Login

In [57]:
try:
    driver = webdriver.Chrome()
    driver.get(login_url)

    login_input    = driver.find_element_by_xpath('//*[@id="tab_section2"]/div/div/form/div[2]/input')
    password_input = driver.find_element_by_xpath('//*[@id="tab_section2"]/div/div/form/div[3]/input')
    button         = driver.find_element_by_xpath('//*[@id="auth-b"]')

    login_input.send_keys(login)
    password_input.send_keys(password)

    button.click()
except:
    print("Already logged in")
print("Ready for parsing")

Variables loaded successfully


### Get all PDF URLs

In [ ]:
with open("links.txt", "r") as file:
    with open('data.csv', 'w', newline='', encoding='utf-8') as csvfile:
        spamwriter = csv.DictWriter(csvfile, fieldnames=["Title", "Category" ,"PDF"])
        spamwriter.writeheader()
        for url in file.readlines():
            print(f"parsing...{url}")
            try:
                data = get_data(url)
                spamwriter.writerow({
                    "Title"    : data["title"],
                    "Category" : data["category"],
                    "PDF"      : data["pdf"]
                })
                print("Success!")
                print("Title: {}".format(data["title"]))
                print("Category: {}".format(data["category"]))
                print("PDF: {}".format(data["pdf"]))
            except Exception as e:
                print("Failed!")
                print(f"Exception: {e}")

### Sort files by categories

In [100]:
with open('data.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if dict(row)["PDF"] is not None:
            titles.append(dict(row)["Title"])
            categories.append(dict(row)["Category"])
            pdfs.append(dict(row)["PDF"])
            
n = len(titles)
for i in range(n):
    if(titles[i]!= ''):
        try:
            result[titles[i]]["categories"].append(categories[i])
            result[titles[i]]["pdfs"].append(pdfs[i])
        except:
            result[titles[i]] = {
                "categories": [],
                "pdfs": []
            }

### Create PDF file with URL

In [ ]:
for key in list(result.keys()):
    path = "pdf/{}".format(key)
    os.mkdir(path)
    print(key)
    for i in range(len(result[key]["pdfs"])):
        try:
            url = result[key]["pdfs"][i]
            wget.download(url,  
                path+'/{}.pdf'.format(result[key]["categories"][i]))
            print("Success: {}".format(result[key]["categories"][i]))
        except Exception as e:
            print("Failed: {}".format(e))
    print("--------------")